# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

# Use the walmart_stock.csv file to Answer and complete the  tasks below!

## Sample Data

```
Date,Open,High,Low,Close,Volume,Adj Close
2012-01-03,59.970001,61.060001,59.869999,60.330002,12668800,52.619234999999996
2012-01-04,60.209998999999996,60.349998,59.470001,59.709998999999996,9593300,52.078475
```

# Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("walmart_stock").getOrCreate()

# Load the Walmart Stock CSV File, have Spark infer the data types.

In [2]:
dir_input_data_path = "./input_data/"
file_input_path = dir_input_data_path + "walmart_stock.csv"

In [3]:
df = spark.read.csv(file_input_path, header=True, inferSchema=True)

# What are the column names?

In [4]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

# What does the Schema look like?

In [5]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



# Print out the first 5 columns.

In [6]:
for row in df.head(5):
    print(row)
    print("\n")

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




# Use describe() to learn about the DataFrame ???

In [7]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [8]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [9]:
from pyspark.sql.functions import format_number

In [10]:
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



# Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [11]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [12]:
df1 = df.withColumn("HV Ratio", df["High"]/df["Volume"])

In [13]:
df1.select("HV Ratio").show(2)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
+--------------------+
only showing top 2 rows



# What day had the Peak High in Price?

In [14]:
df1.orderBy(df["High"].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

# What is the mean of the Close column?

In [15]:
from pyspark.sql.functions import avg

df1.select(avg("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



# What is the max and min of the Volume column?

In [16]:
from pyspark.sql.functions import max, min

df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



# How many days was the Close lower than 60 dollars?

In [17]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [18]:
from pyspark.sql.functions import countDistinct

df.filter(df["Close"]<60).select(countDistinct("Date")).show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                  81|
+--------------------+



In [19]:
df.filter(df["Close"]<60).count()

81

# What percentage of the time was the High greater than 80 dollars ?

In other words, (Number of Days High>80)/(Total Days in the dataset)

In [20]:
from pyspark.sql.functions import count

df.filter(df["High"]>80).count()*100/df.count()

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [21]:
from pyspark.sql.functions import corr

In [22]:
df.select(corr("High", "Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [23]:
df1.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'HV Ratio']

In [24]:
from pyspark.sql.functions import year

df2 = df.withColumn("Year", year(df["Date"]))

In [25]:
df2_year_max = df2.groupBy("Year").max()

In [26]:
df2_year_max.show()

+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|Year|        max(Open)|max(High)| max(Low)|max(Close)|max(Volume)|   max(Adj Close)|max(Year)|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+
|2015|        90.800003|90.970001|    89.25| 90.470001|   80898100|84.91421600000001|     2015|
|2013|        81.209999|81.370003|    80.82| 81.209999|   25683700|        73.929868|     2013|
|2014|87.08000200000001|88.089996|86.480003| 87.540001|   22812400|81.70768000000001|     2014|
|2012|        77.599998|77.599998|76.690002| 77.150002|   38007300|        68.568371|     2012|
|2016|             74.5|75.190002|73.629997| 74.300003|   35076700|        73.233524|     2016|
+----+-----------------+---------+---------+----------+-----------+-----------------+---------+



In [27]:
df2_year_max.select(["Year", "max(High)"]).show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



# What is the average Close for each Calendar Month?

In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [28]:
from pyspark.sql.functions import month

df_month = df.withColumn("Month", month(df["Date"]))

In [29]:
df_month_avg = df_month.groupBy("Month").avg()

In [30]:
df_month_avg.show(3)

+-----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+
|Month|        avg(Open)|        avg(High)|         avg(Low)|       avg(Close)|      avg(Volume)|   avg(Adj Close)|avg(Month)|
+-----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+
|   12|72.87952850943395|73.35566025471698|72.44481152830188|72.84792478301885|7967959.433962264|68.46031040566042|      12.0|
|    1|71.40811884158416|71.97009924752473|70.90425712871289|71.44801958415842|8761851.485148516|65.56887865346533|       1.0|
|    6| 72.5100938962264| 72.9262265471698|72.12198099056603| 72.4953774245283|8303756.603773585|67.43827906603772|       6.0|
+-----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+
only showing top 3 rows



In [31]:
df_month_avg.select(["Month", "avg(Close)"]).orderBy("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!